<a href="https://colab.research.google.com/github/BenjamimOliveira/CN_TP1/blob/main/script/Script2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1 - CN


## Imports

In [105]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import tensorflow.keras.optimizers as optimizers
import time
import zipfile
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load data sources

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

start_time = time.time()

# Copy data (.zip file)
!cp -r 'drive/MyDrive/UMinho/CN/dataset.zip' 'dataset.zip'

# Extract .zip file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset_extr')

print("%s segundos" % ((time.time() - start_time)))

Mounted at /content/drive
33.86333394050598 segundos


## Data Loading

In [92]:
train_dir = '/content/dataset_extr/dataset/train'
test_dir = '/content/dataset_extr/dataset/test'
valid_dir = '/content/dataset_extr/dataset/valid'

imgDataGenerator = False

if imgDataGenerator:
  train_dtgen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
  test_dtgen = ImageDataGenerator(rescale = 1./255)
  valid_dtgen = ImageDataGenerator(rescale = 1./255)

  train_ds = train_dtgen.flow_from_directory(train_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  test_ds = test_dtgen.flow_from_directory(test_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
  valid_ds = valid_dtgen.flow_from_directory(valid_dir,
                                                 target_size = (64, 64),
                                                 batch_size = 32)
else:
  train_ds = tf.keras.preprocessing.image_dataset_from_directory(
      train_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
      test_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
      valid_dir,
      seed=123,
      image_size=(64, 64),
      batch_size=32)
  
  class_names = train_ds.class_names

  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)
  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 35215 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.
Found 1250 files belonging to 250 classes.


## Algoritmo Genético

In [ ]:
##
# Formato do gene [nParesConv2DMaxPooling(entre 1 e 3), learningRate, Momentum, Nesterov, batchSize, Epochs, Accuracy]
#
##

### Gene Aleatório

In [93]:
def random_element():
  gene = []
  
  # -- nPares Conv2D/MaxPooling
  gene.append(random.randint(1,3))

  # -- Learning rate
  gene.append(random.randint(0,4))

  # -- Momentum
  gene.append(random.randint(0, 9)/10)

  # -- Nesterov
  gene.append(random.randint(0,1))

  # -- BatchSize
  gene.append(random.randint(batchSize[0],batchSize[1]))

  # -- Epochs
  gene.append(random.randint(epochs[0], epochs[1]))

  # -- Accuracy
  gene.append(-1)

  return gene

### População Aleatória/Inicial

In [94]:
def random_pool_generator(poolSize):
  pool = []

  for x in range(poolSize):
    pool.append(random_element())

  return pool

### Selecionar "pais" da próxima geração

In [95]:
def select_mating_pool(poolSize, pool):
  top = round(poolSize/2)
  def orderBy(a):
    return a[6]

  pool.sort(reverse=True, key=orderBy)
  return pool[0:top]
  

### Crossover entre 2 elementos

In [96]:
def mix2elements(elemento1, elemento2):
  # tamanho do gene
  size = len(elemento1) - 1
  # elementos que irão ser cruzados
  crossElements = []
  for x in range(3):
    rand = random.randint(0, size-1)
    while (rand in crossElements):
      rand = random.randint(0, size-1)
    crossElements.append(rand)

  elementoFilho = []
  aux = 0

  while aux < size:
    if aux in crossElements:
      elementoFilho.append(elemento1[aux])
    else:
      elementoFilho.append(elemento2[aux])
    aux += 1
  # indica que ainda não foi treinado  
  elementoFilho.append(-1)
  return elementoFilho

### Selecionar que elementos vão fazer crossover

In [97]:
def crossover(poolSize, pool):
  currentPool = poolSize

  # mistura o primeiro com o último
  pool.append(mix2elements(pool[0], pool[-1]))

  aux = 0
  for i in range(int(round(poolSize/2))-1):
    pool.append(mix2elements(pool[aux], pool[aux+1]))
  return pool

### Mutação de um elemento

In [98]:
def mutate(gene, numberOfMutations):
  size = len(gene) - 1
  mut = []
  for x in range(numberOfMutations):
    rand = random.randint(0, size-1)
    while (rand in mut):
      rand = random.randint(0, size-1)
    mut.append(rand)
  
  element = random_element()
  for x in mut:
    gene[x] = element[x]
  
  return gene
  

### Selecionar elementos que irão sofrer mutações

In [99]:
def mutation(pool, chance):
  chance = int(round(chance * 100))
  count = 0
  for gene in pool:
    rand = random.randint(0,100)
    # Decide se um gene vai ou não ser mutado
    if gene[-1] == -1:
      if rand < chance:
        gene = mutate(gene, random.randint(1,2))
      
  return pool

### Gerador de CNN (segundo o gene)

In [103]:
def model_generator(nPares):
  mod = []
  mod.append(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(64, 64, 3)))
  if nPares >= 1: 
    mod.append(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    mod.append(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 2:
    mod.append(layers.Conv2D(64, 3, padding='same', activation='relu'))
    mod.append(layers.Conv2D(64, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  if nPares >= 3:
    mod.append(layers.Conv2D(32, 3, padding='same', activation='relu'))
    mod.append(layers.Conv2D(32, 3, padding='same', activation='relu'))
    mod.append(layers.MaxPooling2D())
  mod.append(layers.Flatten())
  mod.append(layers.Dense(128, activation='relu'))
  mod.append(layers.Dense(num_classes))
  model = Sequential(mod)
  return model

### Conversor de gene em acurácia (treino da cnn com o gene)

In [111]:
def gene_converter(gene):
  # -- MODELO
  model = model_generator(gene[0])
  # -- COMPILAR MODELO
  if gene[3] == 0:
    nest = False
  else:
    nest = True
  opt = optimizers.SGD(learning_rate=learningRate(gene[1]), momentum=gene[2], nesterov=nest)
  # Experimentar loss='sparse_categorical_crossentropy'
  if imgDataGenerator:
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  else:
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  
  es = EarlyStopping(monitor='val_accuracy', patience=3)
  model.compile(optimizer=opt,
                loss=loss,
                metrics=['accuracy'])
  # -- TREINAR MODELO
  epochs=1
  start_time = time.time()
  with tf.device('/device:GPU:0'):
    history = model.fit(
      train_ds,
      validation_data=valid_ds,
      batch_size=gene[4],
      callbacks=[es],
      epochs=gene[5]
    )
  # -- AVALIAR MODELO
  results = model.evaluate(test_ds)
  return results[1]

### Algoritmo Final

In [ ]:
# --------- Parametros chave ---------
def learningRate(lr): return 1/(10 ** lr)
batchSize = [8, 64]
epochs = [4, 15]
num_classes = 250
poolSize = 8
mutationChance = 0.4
geracoes = 2
# ------------------------------------

pool = random_pool_generator(poolSize)

for x in range(geracoes):
  print("----------------------------------")
  print("GERAÇÃO ", x)
  print("----------------------------------")
  count = 1
  # treina a geração
  for gene in pool:
    print("Geração {ger} - Gene {gen}".format(ger=x, gen=count))
    print("Gene: ", gene)
    count += 1
    # -- se ainda não tiver sido treinado numa geração passada então treina
    if gene[6] == -1:
      gene[6] = gene_converter(gene=gene)  
  # seleciona metade do poolsize para se reproduzir
  pool = select_mating_pool(poolSize, pool)
  pool = crossover(poolSize, pool)
  pool = mutation(pool, mutationChance)

count = 1
for gene in pool:
    print("Geração {ger} - Gene {gen}".format(ger=geracoes-1, gen=count))
    print("Gene: ", gene)
    count += 1
    # -- se ainda não tiver sido treinado numa geração passada então treina
    if gene[6] == -1:
      gene[6] = gene_converter(gene=gene)  

----------------------------------
GERAÇÃO  0
----------------------------------
Geração 0 - Gene 1
Gene:  [2, 0, 0.9, 0, 16, 12, -1]
Epoch 1/12
1101/1101 [==============================] - 18s 16ms/step - loss: 16819165993112.5957 - accuracy: 0.0051 - val_loss: 5.6047 - val_accuracy: 0.0040
Epoch 2/12
1101/1101 [==============================] - 17s 15ms/step - loss: 5.5835 - accuracy: 0.0048 - val_loss: 5.6130 - val_accuracy: 0.0040
Epoch 3/12
1101/1101 [==============================] - 17s 15ms/step - loss: 5.5824 - accuracy: 0.0054 - val_loss: 5.6238 - val_accuracy: 0.0040
Epoch 4/12
40/40 [==============================] - 1s 22ms/step - loss: 5.6197 - accuracy: 0.0040
Geração 0 - Gene 2
Gene:  [3, 1, 0.6, 0, 38, 7, -1]
Epoch 1/7
1101/1101 [==============================] - 18s 16ms/step - loss: 5.5159 - accuracy: 0.0067 - val_loss: 5.5333 - val_accuracy: 0.0040
Epoch 2/7
1101/1101 [==============================] - 17s 16ms/step - loss: 5.5067 - accuracy: 0.0083 - val_loss: 5.53

In [ ]:
#for x in pool:
#  print(x)
#print(len(pool[0]))
#print(random.randint(0,5))
elemento1 = [1,1,1,1,1,1,-1]
#elemento2 = [2,2,2,2,2,2,-1]
#print(len(elemento1)-1)

#pool = random_pool_generator(poolSize)
#print(pool)
#pool = mutation(pool, 0.9)

#gene = mutate(elemento1, 2)
#print(gene)

#print(elemento1[-1])
for x in pool:
  print(x)
print("Geração {ger} - Gene {gen}".format(ger="a", gen="b"))

[2, 2, 0.8, 0, 64, 7, 0.5712000131607056]
[2, 2, 0.8, 0, 62, 7, 0.5415999889373779]
[2, 2, 0.7, 0, 64, 7, 0.5167999863624573]
[2, 2, 0.7, 0, 55, 6, 0.5063999891281128]
[2, 2, 0.8, 0, 55, 6, 0.548799991607666]
[2, 4, 0.5, 0, 64, 7, 0.004000000189989805]
[2, 2, 0.8, 0, 64, 7, 0.5784000158309937]
[2, 3, 0.8, 0, 62, 14, 0.3935999870300293]
Geração a - Gene b


# Experiências

In [89]:
from keras.applications.vgg16 import VGG16

def getVGG16Model(lastFourTrainable=False):
  vgg_model = VGG16(weights='imagenet', input_shape=(224,224,3), include_top=True)

  # Make all layers untrainable
  for layer in vgg_model.layers:
      layer.trainable = False

  # Add fully connected layer which have 1024 neuron to VGG-16 model
  output = vgg_model.output
  output = layers.Flatten(name='new_flatten')(output)
  output = layers.Dense(units=1024, activation='relu', name='new_fc')(output)
  output = layers.Dense(units=250, activation='softmax')(output)
  vgg_model = keras.Model(vgg_model.input, output)


  # Compile VGG-16 model
  vgg_model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  vgg_model.summary()

  return vgg_model

def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=1):
  conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
  if fine_tune > 0:
    for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
      for layer in conv_base.layers:
            layer.trainable = False


    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
  top_model = conv_base.output
  top_model = layers.Flatten(name="flatten")(top_model)
  top_model = layers.Dense(4096, activation='relu')(top_model)
  top_model = layers.Dense(1072, activation='relu')(top_model)
  top_model = layers.Dropout(0.2)(top_model)
  output_layer = layers.Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
  model = keras.Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
  model.compile(optimizer=optimizer, 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
  return model

#gene[6] = gene_converter2(gene=gene) 
#print(gene)

b = getVGG16Model()

with tf.device('/device:GPU:0'):
  vgg_history = b.fit(train_ds,
                            epochs=30,
                            batch_size=32,
                            validation_data=valid_ds)

Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

KeyboardInterrupt: ignored